In [ ]:
# Step 1: Import Libraries
import pandas as pd

In [ ]:
# Step 2: Load Required CSV Files
orders = pd.read_csv('../data/olist_orders_dataset.csv')
customers = pd.read_csv('../data/olist_customers_dataset.csv')
reviews = pd.read_csv('../data/olist_order_reviews_dataset.csv')
payments = pd.read_csv('../data/olist_order_payments_dataset.csv')

In [ ]:
# Step 3: Create Churn Labels
customer_orders = pd.merge(orders, customers, on='customer_id', how='left')
customer_orders['order_purchase_timestamp'] = pd.to_datetime(customer_orders['order_purchase_timestamp'])
latest_date = customer_orders['order_purchase_timestamp'].max()
customer_last_order = customer_orders.groupby('customer_unique_id')['order_purchase_timestamp'].max().reset_index()
customer_last_order['days_since_last_order'] = (latest_date - customer_last_order['order_purchase_timestamp']).dt.days
customer_last_order['churned'] = (customer_last_order['days_since_last_order'] > 90).astype(int)

In [ ]:
# Step 4: Feature Engineering
orders_full = pd.merge(orders, customers, on='customer_id', how='left')
order_payment = pd.merge(orders_full, payments, on='order_id', how='left')
merged_reviews = pd.merge(orders_full, reviews, on='order_id', how='left')

# Total orders per customer
total_orders = orders_full.groupby('customer_unique_id')['order_id'].nunique().reset_index(name='total_orders')

# Average review score
avg_review_score = merged_reviews.groupby('customer_unique_id')['review_score'].mean().reset_index(name='avg_review_score')

# Average order value
order_payment['payment_value'] = order_payment['payment_value'].astype(float)
avg_order_value = order_payment.groupby('customer_unique_id')['payment_value'].mean().reset_index(name='avg_order_value')

# Delivery time (days)
orders_full['order_delivered_customer_date'] = pd.to_datetime(orders_full['order_delivered_customer_date'])
orders_full['order_purchase_timestamp'] = pd.to_datetime(orders_full['order_purchase_timestamp'])
orders_full['delivery_time'] = (orders_full['order_delivered_customer_date'] - orders_full['order_purchase_timestamp']).dt.days
delivery_time = orders_full.groupby('customer_unique_id')['delivery_time'].mean().reset_index(name='avg_delivery_time')

In [ ]:
# Step 5: Combine All Features
features = customer_last_order.copy()
features = features.merge(total_orders, on='customer_unique_id', how='left')
features = features.merge(avg_review_score, on='customer_unique_id', how='left')
features = features.merge(avg_order_value, on='customer_unique_id', how='left')
features = features.merge(delivery_time, on='customer_unique_id', how='left')

# Preview the final feature set
features.head()